In [14]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from typing import List, Tuple
from icecream import ic
import random

logging.basicConfig(level=logging.DEBUG)

In [15]:
CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


In [16]:
def path_cost(path: List[int]) -> float:
	cost = 0
	for i in range(len(path) - 1):
		cost += DIST_MATRIX[path[i], path[i + 1]]
	return cost


In [17]:
def tweak(path: List[int]) -> List[int]:
    """ Esegue l'ottimizzazione 2-opt sul percorso dato. """
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1:  # Evita di invertire due punti consecutivi
                    continue
                new_path = path[:]
                new_path[i:j] = reversed(path[i:j])  # Inverti la sottosequenza
                if path_cost(new_path) < path_cost(path):
                    path = new_path
                    improved = True
    return path
# tweak with randomness
MAX_ITER = 10000
def tweak2(path: List[int])-> List[int]:
    
    for _ in range(MAX_ITER):
    
        #choose two random point to reverse (from 1 to n-2)
        #i > j
        i, j = random.sample(range(1, len(path)-1), 2)
        new_path = path[:]
        #reverse the subsequence
        new_path[i:j] = reversed(path[i:j])
        if path_cost(new_path) < path_cost(path):
            path = new_path
                
                        
    return path

	

# Knock Knock Neighbor Algorithm (KKN)

In [18]:

def KKN(start_city: str) -> Tuple[List[int], float]:
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    path = [start_city_index]
    visited = set(path)
    current_city = start_city_index

    while len(visited) < len(CITIES):
        nearest_cities = []
        nearest_distance = float('inf')

        # Trova le città più vicine
        for city_index in range(len(CITIES)):
            if city_index not in visited:
                distance = DIST_MATRIX[current_city, city_index]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_city = city_index

        path.append(nearest_city)
        visited.add(nearest_city)
        current_city = nearest_city

    path.append(start_city_index)  # ritorna alla partenza

    # Ottimizzazione locale: 2-opt
    path = tweak(path)
    

    cost = path_cost(path)
    return path, cost



In [19]:
start_city = "Syracuse"  # Cambia con la città desiderata
(p, c) = KKN(start_city)
print(" Miglior percorso trovato:", [CITIES['name'][i] for i in p])
print(" Costo del miglior percorso:", c, "km")


 Miglior percorso trovato: ['Syracuse', 'Catania', 'Palermo', 'Cagliari', 'Sassari', 'Latina', 'Rome', 'Terni', 'Perugia', 'Florence', 'Prato', 'Leghorn', 'Genoa', 'Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Trento', 'Bolzano', 'Trieste', 'Venice', 'Padua', 'Vicenza', 'Verona', 'Brescia', 'Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Ravenna', 'Forlì', 'Rimini', 'Ancona', 'Pescara', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Salerno', 'Giugliano in Campania', 'Naples', 'Messina', 'Reggio di Calabria', 'Syracuse']
 Costo del miglior percorso: 4379.343649440914 km
